## RCS Python Closures and Generators

In [68]:
## Closures - binding variables from outer function in the inner function
## Technically - function gets stored with its enviroment(bound variables)
### Can also think of preserving certain state

In [1]:
# remember this function?
def add_factory(x):
    def add(y):
        return y + x 
    return add # upon return free variable x gets bound in the add function


In [2]:
add5 = add_factory(5)
# 5 is bound inside add5 now,
add5(10)

15

In [3]:
type(add5.__closure__)

tuple

In [4]:
[x for x in add5.__closure__]

[<cell at 0x10bb9f2e8: int object at 0x108cb55d0>]

In [5]:
len(add5.__closure__)

1

In [6]:
int(add5.__closure__[0])

TypeError: int() argument must be a string, a bytes-like object or a number, not 'cell'

In [7]:
dir(add5.__closure__[0])

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'cell_contents']

In [8]:
add5.__closure__[0].cell_contents

5

In [ ]:
## Voila!! We get what we expected to get!

In [ ]:
## Remember __closure__ is a tuple so we do not get to mutate this!

In [ ]:
So how about more values stored?

In [9]:
def add2_fact(x, y):
    return lambda z: z+x+y

In [10]:
a10n20 = add2_fact(10,20)

In [11]:
a10n20(50)

80

In [13]:
len(a10n20.__closure__)

2

In [14]:
[x.cell_contents for x in a10n20.__closure__]

[10, 20]

In [ ]:
One last closure example:

In [15]:
def outer(x):
    a = 20
    def inner(y):
        print(f'x:{x}')
        print(f'a:{a}')
        print(f'y:{y}')
        ## x += 15 We can't change the argument that was bound from outside argument
        ## a += 15 We can't change the a that was bound from outside function
        return a+x+y
    return inner

In [16]:
axy = outer(10)

In [17]:
axy(5)

x:10
a:20
y:5


35

In [20]:
axy(5)

x:10
a:20
y:5


35

In [32]:
[x.cell_contents for x in axy.__closure__]

[20, 10]

## What if we want rebind(assign new values) to variables coming from outer scope?
### In languages like Javascript you can do it, so Python should be able to, right?
### Solution: Python3 nonlocal modifier inside inner function 

In [57]:
# https://docs.python.org/3/reference/simple_stmts.html#the-nonlocal-statement
# 7.13. The nonlocal statement

# The nonlocal statement causes the listed identifiers to refer to previously bound variables in the nearest enclosing scope excluding globals. 
# This is important because the default behavior for binding is to search the local namespace first. The statement allows encapsulated code to rebind variables outside of the local scope besides the global (module) scope.

# Names listed in a nonlocal statement, unlike those listed in a global statement, must refer to pre-existing bindings in an enclosing scope (the scope in which a new binding should be created cannot be determined unambiguously).

# Names listed in a nonlocal statement must not collide with pre-existing bindings in the local scope.

In [21]:
def makeCounter():
    count = 0
    def f():
        nonlocal count
        count +=1
        return count
    return f


In [22]:
a = makeCounter()

In [23]:
a()

1

In [24]:
a()

2

In [25]:
a()

3

In [26]:
print(a(),a(),a())

4 5 6


In [27]:
[a()for x in range(10)]

[7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [28]:
dir(a())

['__abs__',
 '__add__',
 '__and__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '__xor__',
 'bit_length',
 'conjugate',
 'denominator',
 'from_bytes',
 'imag',
 'numerator',
 'real',
 'to_bytes']

In [29]:
def makeAdjCounter(x):
    count = 0
    def f():
        nonlocal count  # without nonlocal we could reference count but couldn't modify it!
        count += x
        return count
    return f

In [39]:
b = makeAdjCounter(2)
c = makeAdjCounter(4)

In [31]:
print(b(),b(),b())

2 4 6


In [40]:
print(c(),c(),c(),c())

4 8 12 16


In [41]:
[x.cell_contents for x in c.__closure__]

[16, 4]

In [58]:
# Result count is hidden from us, but by calling function we can modify its value.


In [78]:
## Another older way was to create some structure(List, Class, Dictionary) inside outer function whose members could be modified by innner


In [54]:
def makeAdjList():
    holder=[1,0,2,3] # old method not recommended anymore!
    def f():
        holder[0] +=1
        return holder[0]
    return f

In [55]:
d = makeAdjList()


In [56]:
print(d(),d(),d())

2 3 4


### Most Python answer is to use generators for persisting some sort of iterable state

## What the heck is a Generator ?

###  A Python generator is a function which returns a generator iterator (just an object we can iterate over) by calling yield

* KEY POINT: generator functions use **yield** instead of **return**
* in Python 3 we use next(generatorName) to obtain next value

In [57]:
def makeNextGen(current):
    while True: ##This means our generator will never run out of values...
        current += 1
        yield current
    

In [58]:
numGen = makeNextGen(30)

In [62]:
for i in range(15):
    print(next(numGen))  # This is for Python 3.x ,  in Python 2.x it was numGen.next()

36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [93]:
## We can do even better and make an adjustable increment

In [63]:
def makeNextGenInc(current, inc):
    while True: 
        current += inc
        yield current


In [127]:
def smallYield():
    yield 1
    yield 2
    yield 99
    yield 5
    

In [128]:
smallGen = smallYield()

In [133]:
next(smallGen)

StopIteration: 

In [64]:
numGen10 = makeNextGenInc(200, 10)

In [67]:
[next(numGen10) for x in range(10)]

[410, 420, 430, 440, 450, 460, 470, 480, 490, 500]

In [101]:
## Now the above is Pythonic approach to the problem!

In [116]:
### Then there is a generator expression 
## The whole point is have a lazy evaluation (ie no need to have everything at once in memory)


In [97]:
gen = (i+10 for i in range(10))


In [98]:
list(gen)

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [70]:
type(gen)

generator

In [71]:
list(gen)

[]

In [ ]:
# list(i+10 for i in range(10)) == [i+10 for i in range(10)]

In [112]:
gen = (i+10 for i in range(10))

In [91]:
for el in gen:
    print(el)

10
11
12
13
14
15
16
17
18
19


In [92]:
for g in gen:
    print(g)

In [164]:
# You see what is going on?!

In [104]:
gen_exp = (x ** 2 for x in range(10) if x % 2 == 0)
type(gen_exp)

generator

In [105]:
for x in gen_exp:
    print(x)

0
4
16
36
64


In [103]:
glist = list(gen)
glist

[]

In [113]:
[next(gen) for x in range(5)]

[10, 11, 12, 13, 14]

In [150]:
yes_expr = ('yes' for _ in range(10))

In [152]:
list(yes_expr)

['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes']

In [114]:
## Challenge how to make an infinite generator with a generator expression?

In [114]:
import itertools

In [115]:
genX = (i*5 for i in itertools.count(start=0, step=1))


In [116]:
[next(genX) for x in range(20)]

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]

In [123]:
import random
genY = (i*10+random.randrange(10) for i in itertools.count(start=0, step=1))

In [126]:
[next(genY) for x in range(10)]

[207, 213, 229, 238, 241, 257, 261, 273, 282, 294]

In [ ]:
d

In [168]:
## Be very careful with infinite generators, calling list on infinite generator not recommended!

In [129]:
## Of course we should generally have a pretty good idea of maximum number of generations needed

### Difference between Python's Generators and Iterators
* iterators is more general, covers all generators

From Official Docs: Python’s generators provide a convenient way to implement the iterator protocol. If a container object’s __iter__() method is implemented as a generator, it will automatically return an iterator object (technically, a generator object) supplying the __iter__() and next()

## A Generator is an Iterator
### Specifically, generator is a subtype of iterator.

Conceptually:
Iterators are about various ways to loop over data, generators generate the data on the fly

## Homework
### Write a generator to yield cubes (forever!)
### Write a generator to yield Fibonacci numbers(first 1000)

* Generator Functions ok to use here

In [ ]:
# Hint use yield

In [ ]:
# Extra Credit! write generator expression for first 500 cubes that are made from even numbers